In [144]:
import time
time.tics = []

def tic ():
    time.tics.append (time.time ())
    
def toc (prefix=''):
    print (prefix + 'Elapsed: {:.2f} sec'.format (time.time () - time.tics.pop ()))

In [145]:
input_file = 'uf20-91/uf20-01.cnf'

In [146]:
from os.path import basename
input_base = basename (input_file).split ('.')[0]

In [147]:
with open (input_file, 'r') as f:
    raw_lines = f.readlines ()
print (raw_lines[:5])
print ('...')
print (raw_lines[-5:])
print ('length: %d' % len (raw_lines))

['c This Formular is generated by mcnf\n', 'c\n', 'c    horn? no \n', 'c    forced? no \n', 'c    mixed sat? no \n']
...
['12 -2 17 0\n', '4 -16 -5 0\n', '%\n', '0\n', '\n']
length: 102


In [148]:
clauses = []

for line in raw_lines:
    try:
        clause = [int (x) for x in line.split ()][:3]
        if len (clause) == 3:
            clauses.append (clause)
    except:
        pass

print (clauses[:5])
print ('...')
print (clauses[-5:])
print ('length: %d' % len (clauses))

[[4, -18, 19], [3, 18, -5], [-5, -8, -15], [-20, 7, -16], [10, -13, -7]]
...
[[-20, -18, 11], [-9, 1, -5], [-19, 9, 17], [12, -2, 17], [4, -16, -5]]
length: 91


In [149]:
flattened = [x for clause in clauses for x in clause]
print (flattened[:15])
print ('...')
print (flattened[-15:])
print ('length: %d' % len (flattened))

[4, -18, 19, 3, 18, -5, -5, -8, -15, -20, 7, -16, 10, -13, -7]
...
[-20, -18, 11, -9, 1, -5, -19, 9, 17, 12, -2, 17, 4, -16, -5]
length: 273


In [150]:
uniques = set (flattened)
print (*uniques)
print ('length: %d' % len (uniques))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 -10 -9 -8 -7 -6 -5 -4 -3 -2 -1
length: 40


In [151]:
# Directly evaluate clauses for given assignment
def verify (assignment, clauses):
    for clause in clauses:
        for x in clause:
            result = (x > 0) ^ assignment[abs (x)-1]
            if result:
                break
        if not result:
            return False
    
    return True

In [152]:
# Naively checking all possible truth assignments
from itertools import product

valid = []

tic ()
for i, assignment in enumerate (product (range (2), repeat=len ({abs (x) for x in uniques}))):
    if verify (assignment, clauses):
        valid.append (assignment)
toc ()

print ('Valid assignments found after {} iterations: {}.'.format (i+1, len (valid)))
print (*valid, sep='\n')

Elapsed: 3.08 sec
Valid assignments found after 1048576 iterations: 8.
(0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0)
(0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0)
(1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0)


In [153]:
from collections import defaultdict

value2indices = defaultdict (list)

for i,x in enumerate (flattened):
    value2indices[x].append (i)

# Print result
for i,(k,v) in enumerate (value2indices.items ()):
    if i < 5 or i >= len (value2indices) - 5:
        print ('{:3d} : {}'.format (k, v))
    if i==4:
        print ('...')

print ('length: {}'.format (len (value2indices)))

  4 : [0, 46, 50, 80, 152, 153, 182, 224, 270]
-18 : [1, 36, 75, 206, 210, 233, 234, 259]
 19 : [2, 19, 81, 124, 176, 205, 227]
  3 : [3, 148, 235, 244, 247]
 18 : [4, 27, 43, 85, 181]
...
 20 : [67, 77, 78, 105, 129, 140, 198, 222, 252]
 -2 : [76, 172, 177, 188, 226, 243, 248, 249, 268]
 -4 : [86, 117, 133, 212]
 -1 : [87, 146, 170, 199, 218]
 16 : [100, 127, 147]
length: 40


In [154]:
from itertools import combinations

adjacencies = defaultdict (set)

for i, clause in enumerate (clauses):
    base = 3 * i
    for j, literal  in enumerate (clause):
        idx = base + j
        
        # A literal is adjacent to literals that are its complement...
        complement = value2indices[-literal]
        adjacencies[idx].update (complement)
        
        # ...and to the Literals in the same clause
        adjacencies[idx].update ({base + offset for offset in range (3)})
        adjacencies[idx].remove (idx)

# Print result
for i,(k,v) in enumerate (adjacencies.items ()):
    if i < 5 or i >= len (adjacencies) - 5:
        print ('{:3d} : {}'.format (k, v))
    if i==4:
        print ('...')
print ('length: %d' % len (adjacencies))

  0 : {1, 2, 212, 117, 86, 133}
  1 : {0, 2, 4, 43, 85, 181, 27}
  2 : {0, 257, 1, 197, 39, 264, 108, 44, 143, 89, 220}
  3 : {242, 4, 5, 250, 155, 30}
  4 : {1, 259, 36, 3, 5, 233, 234, 75, 206, 210}
...
268 : {231, 267, 60, 269}
269 : {256, 34, 68, 230, 202, 74, 267, 268, 88}
270 : {272, 212, 117, 86, 133, 271}
271 : {272, 147, 100, 270, 127}
272 : {194, 196, 20, 189, 270, 271}
length: 273


In [161]:
# Returns list of BFS layers from given (full, unabridged) adjacency list <adj>

def bfs (adj, start=None, append_empty=True):
    # Precondition:
    assert adj
    
    # Track remaining unprocessed nodes
    remaining = {x for x in adj.keys ()}
    
    # Decide BFS starting root node
    if start in remaining:
        remaining.remove (start)
    else:
        start = remaining.pop ()
    
    # BFS layers and record of seen nodes
    layers = [{start}]
    seen = {start}
    
    while remaining:
        # Prepare to construct next_layer
        prev_layer = layers[-1]
        next_layer = set ()
        
        # Populate next_layer
        for node in prev_layer:
            next_layer |= adj[node]
        next_layer -= seen
        
        # Record next_layer
        remaining -= next_layer
        seen |= next_layer
        # Note: Append even if next_layer is empty.
        # An empty set will separate layers of
        # different connected components
        layers.append (next_layer)
        
        # Start new connected component
        if not next_layer and remaining:
            if not append_empty:
                layers.pop ()
            start = remaining.pop ()
            seen.add (start)
            layers.append ({start})
        
        # Remove final trailing empty set (applicable iff append_empty option set)
        elif not next_layer:
            layers.pop ()
            
    return layers

In [156]:
bfs_layers = bfs (adjacencies)

print (bfs_layers[:2])
print ('...')
print (bfs_layers[-1:])
print ('lengths: {}'.format ([len (layer) for layer in bfs_layers]))

[{0}, {1, 2, 212, 117, 86, 133}]
...
[{7, 10, 267, 268, 31, 32, 33, 35, 51, 60, 67, 72, 102, 104, 105, 115, 120, 136, 159, 160, 163, 164, 166, 167, 172, 173, 186, 188, 192, 198, 237, 241, 246, 248, 249, 252}]
lengths: [1, 6, 29, 76, 125, 36]


In [157]:
from functools import reduce
from operator import mul

abridged_adj = {k: {i for i in v if i > k} for k,v in adjacencies.items ()}

# Print result
for i,(k,v) in enumerate (adjacencies.items ()):
    if i < 5 or i >= len (adjacencies) - 5:
        print ('{:3d} : {}'.format (k, v))
    if i==4:
        print ('...')
    
print ('length: %d' % len (abridged_adj))
print ()

# Compare average lengths
old, new = sum (len (s) for s in adjacencies.values ()), sum (len (s) for s in abridged_adj.values ())
old, new = old/len (adjacencies), new/len (adjacencies)
print ('Arithmetic Average Lengths:')
print ('old : {:.2f}, new : {:.2f}, factor improved: {:.2f}'.format (old, new, old/new))
print ()

old, new = reduce (mul, (1+len (s) for s in adjacencies.values ()), 1), reduce (mul, (1+len (s) for s in abridged_adj.values ()), 1)
old, new = old**(1/len (adjacencies)), new**(1/len (adjacencies))
print ('Geometric Average Lengths:')
print ('old : {:.2f}, new : {:.2f}, factor improved: {:.2f}'.format (old, new, old/new))

  0 : {1, 2, 212, 117, 86, 133}
  1 : {0, 2, 4, 43, 85, 181, 27}
  2 : {0, 257, 1, 197, 39, 264, 108, 44, 143, 89, 220}
  3 : {242, 4, 5, 250, 155, 30}
  4 : {1, 259, 36, 3, 5, 233, 234, 75, 206, 210}
...
268 : {231, 267, 60, 269}
269 : {256, 34, 68, 230, 202, 74, 267, 268, 88}
270 : {272, 212, 117, 86, 133, 271}
271 : {272, 147, 100, 270, 127}
272 : {194, 196, 20, 189, 270, 271}
length: 273

Arithmetic Average Lengths:
old : 8.32, new : 4.16, factor improved: 2.00

Geometric Average Lengths:
old : 8.97, new : 4.46, factor improved: 2.01


In [162]:
# Check whether a combination of nodes is valid based on adjacencies
from functools import reduce

def is_valid (combo, adj):
    for i in combo:
        if not adj[i].isdisjoint (combo):
            return False, None
    
    adjacent = reduce (set.__or__, (adj[i] for i in combo))
    return True, adjacent

In [159]:
# Recursively forms nested BFS layers from each BFS layer
def recursive_bfs (adj, **kwargs):
    # Precondition:
    assert adj
    
    # Run BFS initially to get layers
    layers = bfs (adj, **kwargs)

    # Recursively decompose each BFS layer
    for i, layer in enumerate (layers):
        # Prepare layer-local adj
        # (i.e. restrict edges to nodes within this layer)
        local_adj = {node: adj[node] & layer for node in layer}
        
        # Done if there is one or fewer nodes
        if len (local_adj) <= 1:
            continue
        
        # Update layer with recursively decomposed result
        else:
            layers[i] = recursive_bfs (local_adj, **kwargs)

    return layers

In [84]:
rbfs_layers = recursive_bfs (adjacencies)

print (rbfs_layers[:2])
print ('...')
print (rbfs_layers[-1:])
print ('lengths: {}'.format ([len (layer) for layer in rbfs_layers]))

[{0}, [{1}, {2}, set(), {133}, set(), {212}, set(), {117}, set(), {86}]]
...
[[{7}, set(), {136}, set(), {10}, set(), {267}, {268}, {60}, [{248}, set(), {249}, set(), {188}, set(), {172}], [{186}, set(), {173}, set(), {246}], [{102}, set(), {166}], [{192}, set(), {33}, set(), {163}, set(), {167}, set(), {104}, set(), {241}], [{35}, set(), {164}], set(), {159}, {160}, set(), {32}, {31}, set(), {51}, set(), {67}, set(), {198}, set(), {72}, set(), {105}, set(), {237}, set(), {115}, set(), {120}, set(), {252}]]
lengths: [1, 10, 38, 45, 28, 38]


In [163]:
# Convert RBFS layers into HTML representation

def render (rbfs_tree, depth=0):
    text = []
    
    # Record seen nodes just to double-check
    seen = set ()

    # Set div orientation and indent level
    indent = '  ' * (depth + 2)
    if depth % 2:
        orientation = 'horizontal'
    else:
        orientation = 'vertical'

    # Render each layer (recursively if needed)
    for i, layer in enumerate (rbfs_tree):
        
        # Connected component divider
        if not layer:
            text.append ('{}<div class="break {}"></div>'.format (indent, orientation))
        
        # Node
        elif isinstance (layer, set):
            index, = [x for x in layer]
            text.append ('{}<div id="node_{}" class="node {}">{}</div>'.format (indent, index, orientation, index))
            seen.update (layer)
            
        # Layer (must recurse deeper)
        else:
            rendered, _ = render (layer, depth=depth+1)
            text.append ('{}<div class="layer {}">\n{}\n{}</div>'.format (indent, orientation, rendered, indent))
            seen.update (_)
    
    # Join rendered layers
    text = '\n'.join (text)
    
    # For outermost level, wrap in container, apply stylesheet
    if depth == 0:
        template = '''\
<!DOCTYPE html>
<html>
<head>
  <title>BFS Decomposition: {}</title>
</head>
<body>
  <div class="container horizontal">
  <canvas></canvas>
{}
  </div>
  <link rel="stylesheet" type="text/css" href="../css/style.css">'
  <script type="module" src="../js/draw_edges.js"></script>
</body>
</html>'''
        text = template.format (input_base, text)
    
    return text, seen

In [166]:
# Displays first <lines> of <filename>, printing trailing ellipsis ('...') if <trail>
def head (filename, lines=5, trail=True):
    if lines is None or lines < 0:
        from itertools import count
        lines = count ()
    else:
        lines = range (lines)
        
    with open (filename, 'r') as f:
        for _ in lines:
            text = f.readline ()
            
            if not text:
                trail = False
                break
                
            print (text, end='')
            
    if trail:
        print ('...')

In [180]:
html_file = 'html/{}.html'.format (input_base)

In [181]:
text, seen = render (rbfs_layers)

print ('Number of seen nodes: {}'.format (len (seen)))

with open (html_file, 'wt') as f:
    f.write (text)

# Confirm html looks good
head (html_file, 15)

Number of seen nodes: 273
<!DOCTYPE html>
<html>
<head>
  <title>BFS Decomposition: uf20-01</title>
</head>
<body>
  <div class="container horizontal">
  <canvas></canvas>
    <div id="node_0" class="node vertical">0</div>
    <div class="layer vertical">
      <div id="node_1" class="node horizontal">1</div>
      <div id="node_2" class="node horizontal">2</div>
      <div class="break horizontal"></div>
      <div id="node_133" class="node horizontal">133</div>
      <div class="break horizontal"></div>
...


In [182]:
edges_file = 'json/{}.json'.format (input_base)

In [183]:
import json
with open (edges_file, 'wt') as f:
    json.dump ({node: sorted (adjacent) for node, adjacent in abridged_adj.items ()}, f, indent=2, sort_keys=True)

head (edges_file, 15)

{
  "0": [
    1,
    2,
    86,
    117,
    133,
    212
  ],
  "1": [
    2,
    4,
    27,
    43,
    85,
...


In [123]:
# Search RBFS layers for valid subsets

from operator import mul
from functools import reduce
from itertools import combinations, product
from math import factorial

def recursive_valid_subsets (layers, registered, depth=0):
    indent = '..' * depth
    
    # Return immediately if reached a leaf node
    if len (layers) == 0:
        print (indent + 'Return immediately from depth {}.'.format (depth))
        return [tuple ()], 1
    elif len (layers) == 1:
        print (indent + 'Return immediately from depth {}.'.format (depth))
        return [tuple (layers)], 1
    
    print (indent + 'Start depth {}.'.format (depth))
    print (indent + '{}.'.format (layers))
    
    # Get all subsets for deeper layers (recursively)
    tic ()
    print (indent + 'Recurse {} deeper layers.'.format (len (layers)))
    layers, num_checked = zip (*(recursive_valid_subsets (layer, registered, depth=depth+1) for layer in layers))
    total = sum (len (layer) for layer in layers)
    print (indent + 'Returning from {} deeper layers. Total number of subsets: {}.'.format (len (layers), total))
    print (indent + 'Product space size: {}.'.format (reduce (mul, (len (layer) for layer in layers), 1)))
    toc (indent)
    
    # Form all compatible new subsets from unions of old subsets (try all combinations from separate layers)
    n = len (layers)
    subsets = [subset for layer in layers for subset in layer]
    tally = count = 0
    length_seen = 1
    tic ()
    for r in range (2, n+1):
        for layer_indices in combinations (range (n), r):
            for combo in product (*(layers[i] for i in layer_indices)):
                combo = reduce (set.union, combo, set ())
                valid, adjacent = is_valid (combo, registered)
                if valid:
                    if depth == 0 and len (combo) > length_seen:
                        length_seen = len (combo)
                        subsets.append (combo)
                    elif depth > 0:
                        subsets.append (combo)
                    tally += 1
                count += 1
    toc (indent)
    
    print (indent + 'End Depth {}.\n'.format (depth) +
           indent + '{} combinations checked.\n'.format (count) +
           indent + 'New subsets formed: {}.\n'.format (tally))
    print (indent[:-2] + '________________________________________________')
           
    return subsets, sum (num_checked) + count

In [121]:
# Note append_empty=False
rbfs_layers_no_gap = recursive_bfs (adjacencies, append_empty=False)

print (rbfs_layers_no_gap[:3])
print ('...')
print (rbfs_layers_no_gap[-1:])
print ('lengths: {}'.format ([len (layer) for layer in rbfs_layers_no_gap]))

[{0}, [{1}, {2}, {133}, {212}, {117}, {86}]]
...
[[{7}, {136}, {10}, {267}, {268}, {60}, [{248}, {249}, {188}, {172}], [{186}, {173}, {246}], [{102}, {166}], [{192}, {33}, {163}, {167}, {104}, {241}], [{35}, {164}], {159}, {160}, {32}, {31}, {51}, {67}, {198}, {72}, {105}, {237}, {115}, {120}, {252}]]
lengths: [1, 6, 22, 28, 21, 24]


In [125]:
registered = abridged_adj.copy ()
tic ()
subsets, count = recursive_valid_subsets (rbfs_layers_no_gap[2], registered)
toc ()
print ('Final result: {} subsets, {} combinations checked.'.format (len (subsets), count))

Start depth 0.
[{257}, {4}, {210}, [{43}, {211}, {85}, {181}, {27}], [{84}, {182}, {44}], {132}, [{134}, {119}], {118}, {264}, {270}, {143}, {152}, {153}, {39}, {46}, {50}, {197}, {80}, {89}, {220}, {224}, {108}].
Recurse 22 deeper layers.
..Return immediately from depth 1.
..Return immediately from depth 1.
..Return immediately from depth 1.
..Start depth 1.
..[{43}, {211}, {85}, {181}, {27}].
..Recurse 5 deeper layers.
....Return immediately from depth 2.
....Return immediately from depth 2.
....Return immediately from depth 2.
....Return immediately from depth 2.
....Return immediately from depth 2.
..Returning from 5 deeper layers. Total number of subsets: 5.
..Product space size: 1.
..Elapsed: 0.00 sec
..Elapsed: 0.00 sec
..End Depth 1.
..26 combinations checked.
..New subsets formed: 26.

________________________________________________
..Start depth 1.
..[{84}, {182}, {44}].
..Recurse 3 deeper layers.
....Return immediately from depth 2.
....Return immediately from depth 2.
....

KeyboardInterrupt: 